# Parallel computing in the notebook

We can use the IPython `ipyparallel` environment for parallel computing right in the notebook.

[Read the docs.](https://ipyparallel.readthedocs.io/en/latest/intro.html)

[To install](https://ipyparallel.readthedocs.io/en/latest/index.html):

    conda install ipyparallel
    ipcluster nbextension enable
    
If that doesn't work, try doing `conda install jupyter` first.

In [3]:
import numpy as np

## A little demo

In [4]:
import ipyparallel as ipp
c = ipp.Client()
c.ids

[0, 1, 2, 3, 4, 5, 6, 7]

In [ ]:
# DirectView
dview = c[:]

In [ ]:
dview.apply_sync(lambda: "Hello world")

In [ ]:
%timeit list(map(lambda x: (x**3.14159)**0.5, range(int(1e6))))

In [ ]:
%timeit list(dview.map_sync(lambda x: (x**3.14159)**0.5, range(int(1e6))))

## `%px` magic

We can do parallel execution easily with a magic:

In [2]:
with c[:].sync_imports():
    import numpy

importing numpy on engine(s)


In [3]:
%px a = numpy.random.rand(4, 4)

In [4]:
%px numpy.linalg.eigvals(a)

Out[0:2]: 
array([ 1.84074705+0.j       ,  0.78606188+0.j       ,
       -0.47007729+0.2532012j, -0.47007729-0.2532012j])

Out[1:2]: array([ 2.13906404, -0.57217535,  0.23099227,  0.02159242])

Out[2:2]: 
array([ 1.67093518+0.j        , -0.21883437+0.44641444j,
       -0.21883437-0.44641444j,  0.15355235+0.j        ])

Out[3:2]: 
array([ 2.18834795+0.j        , -0.28245862+0.j        ,
        0.22598497+0.08537044j,  0.22598497-0.08537044j])

## Function decorator

In [5]:
np.random.seed(1)
layers = np.random.random(int(1e6))

In [6]:
#@dview.parallel()  ## See remark below about this decorator.
def compute_rc(layers):
    """
    Computes reflection coefficients given
    a list of layer impedances.
    """
    uppers = layers[:-1]
    lowers = layers[1:]
    rcs = []
    for pair in zip(lowers, uppers):
        rc = (pair[1] - pair[0]) / (pair[1] + pair[0])
        rcs.append(rc)
    return rcs

In [7]:
def compute_rc_vector(layers):
    layers = np.array(layers)
    uppers = layers[:-1]
    lowers = layers[1:]
    return (lowers - uppers) / (uppers + lowers)

#### list, serial

In [8]:
%timeit compute_rc(layers)

1 loop, best of 3: 532 ms per loop


#### list, parallel

In [9]:
# NB This is the same as using @dview.parallel() to decorate the
# original function when we defined it, as shown in that block.
compute_rc_parallel = dview.parallel()(compute_rc)

In [10]:
%timeit compute_rc_parallel(layers)

100 loops, best of 3: 13.5 ms per loop


#### ndarray, serial

In [11]:
%timeit compute_rc_vector(layers)

100 loops, best of 3: 10.3 ms per loop


#### ndarray, parallel

In [12]:
compute_rc_vector_parallel = dview.parallel()(compute_rc_vector)

In [13]:
%timeit compute_rc_vector_parallel(layers)

10 loops, best of 3: 6 ms per loop


## Parallel list comprehension

Via `scatter` and `gather`. [From the docs](https://ipyparallel.readthedocs.io/en/latest/multiengine.html#scatter-and-gather):

> Sometimes it is useful to partition a sequence and push the partitions to different engines. In MPI language, this is know as scatter/gather and we follow that terminology [...] `scatter()` is from the interactive IPython session to the engines and `gather()` is from the engines back to the interactive IPython session.

We start by scattering the iterable (notice that we have to call list on everything because everything is lazily executed):

In [5]:
dview.scatter('y', range(16))

# And look at it:
list(dview['y'])

[range(0, 4), range(4, 8), range(8, 12), range(12, 16)]

Now we can compute with the pieces, using the 'parallel execution' magic, `%px`:

In [6]:
%px z = [(i**3.14159)**0.5 for i in y]
list(dview['z'])

[[0.0, 1.0, 2.970683691519495, 5.616421346404785],
 [8.824961595059897,
  12.529639852302871,
  16.68461129846666,
  21.255717809934282],
 [26.216169488730305,
  31.544188740351338,
  37.22159676984888,
  43.23290542839072],
 [49.564702683696815, 56.20521948320366, 63.14401424951226, 70.37173672923794]]

Now cast back out to a Python sequence.

In [7]:
z = dview.gather('z')
list(z)

[0.0,
 1.0,
 2.970683691519495,
 5.616421346404785,
 8.824961595059897,
 12.529639852302871,
 16.68461129846666,
 21.255717809934282,
 26.216169488730305,
 31.544188740351338,
 37.22159676984888,
 43.23290542839072,
 49.564702683696815,
 56.20521948320366,
 63.14401424951226,
 70.37173672923794]